In [113]:
import pandas as pd

In [114]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [115]:
mov = pd.read_csv('data/ratings_movies.csv')

In [116]:
movie = mov.copy()
display(movie.head(1))

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [117]:
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
movie['year_release'] = movie['title'].apply(get_year_release)
display(movie.head(1))

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0


In [ ]:
# Если есть пропуски, для преобразования значения 'Int', 'Float'
movie['year_release'] = movie['year_release'].astype('Int64')

In [143]:
movie['date'] = pd.to_datetime(movie['date'])

In [144]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  Int64         
 8   year_reting   0 non-null       object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 7.0+ MB


In [120]:
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя 
# оценка фильмам наибольшая.
#В качестве ответа укажите идентификатор этого пользователя.
#Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), 
# можно воспользоваться методом agg() на сгруппированных данных.
mask1 = movie.groupby('userId')['rating'].agg(['count', 'mean'])
mask2 = mask1.sort_values(by=['count', 'mean'], ascending=[True, False])
print(mask2)

        count      mean
userId                 
53         20  5.000000
595        20  4.200000
189        20  4.100000
569        20  4.000000
278        20  3.875000
...       ...       ...
274      1346  3.235884
448      1864  2.847371
474      2108  3.398956
599      2478  2.642050
414      2698  3.391957

[610 rows x 2 columns]


In [139]:
# Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
#Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
mask1 = movie['year_release'] == 2018
mask2 = movie[mask1].groupby('genres')['rating'].agg(['mean', 'count'])
mask22 = mask2['count'] > 10
mask3 = mask2[mask22].sort_values(by=['mean', 'count'], ascending=[False, True])
print(mask3)

                             mean  count
genres                                  
Action|Adventure|Sci-Fi  3.928571     14
Action|Comedy|Sci-Fi     3.875000     12


In [157]:
# Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость 
# среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:
movie['year_reting'] = movie['date'].dt.year
movei_reting = movie.pivot_table(
    index='year_reting', 
    columns='genres', 
    values='rating', 
    aggfunc='mean')
movei_reting['Comedy']

year_reting
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64